# Parsimonious flux balance analysis

We will use `parsimonious_flux_balance_analysis` to find the optimal
flux distribution in the *E. coli* "core" model.

TODO pFBA citation

In [1]:
using COBREXA

download_model(
    "http://bigg.ucsd.edu/static/models/e_coli_core.json",
    "e_coli_core.json",
    "7bedec10576cfe935b19218dc881f3fb14f890a1871448fc19a9b4ee15b448d8",
)

[ Info: using cached `e_coli_core.json'


"e_coli_core.json"

next, load the necessary packages

In [2]:
import JSONFBCModels
import Clarabel # can solve QPs

model = load_model("e_coli_core.json") # load the model

JSONFBCModels.JSONFBCModel(#= 95 reactions, 72 metabolites =#)

Use the convenience function to run standard pFBA on

In [3]:
solution = parsimonious_flux_balance_analysis(
    model;
    optimizer = Clarabel.Optimizer,
    settings = [silence],
)

ConstraintTrees.Tree{Float64} with 4 elements:
  :flux_stoichiometry     => ConstraintTrees.Tree{Float64}(#= 72 elements =#)
  :fluxes                 => ConstraintTrees.Tree{Float64}(#= 95 elements =#)
  :objective              => 0.873922
  :parsimonious_objective => 11414.2

## Linear version

TODO

In [4]:
import GLPK

linear_solution =
    linear_parsimonious_flux_balance_analysis(model; optimizer = GLPK.Optimizer)

ConstraintTrees.Tree{Float64} with 7 elements:
  :directional_flux_balance => ConstraintTrees.Tree{Float64}(#= 95 elements =#)
  :flux_stoichiometry       => ConstraintTrees.Tree{Float64}(#= 72 elements =#)
  :fluxes                   => ConstraintTrees.Tree{Float64}(#= 95 elements =#)
  :fluxes_forward           => ConstraintTrees.Tree{Float64}(#= 95 elements =#)
  :fluxes_reverse           => ConstraintTrees.Tree{Float64}(#= 95 elements =#)
  :objective                => 0.873922
  :parsimonious_objective   => 518.422

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*